In [1]:
# IMPORTING LIBRARIES

import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Feature selection and engineering
from sklearn.feature_selection import RFE

#Model evaluation
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Example usage
if __name__ == "__main__":
    # Load data
    df = pd.read_csv("wells_data_cleaned.csv")

    # Initialize DataProcessor
    processor = DataProcessor(df, target_column="status_group")

    # Process the data
    processor.process_data()

    # Access processed data
    X_train = processor.X_train
    X_test = processor.X_test
    y_train = processor.y_train_encoded
    y_test = processor.y_test_encoded

    print("Processed training and testing data are ready.")